In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, \
    recall_score, f1_score, roc_auc_score, matthews_corrcoef, roc_curve
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
plt.rcParams["figure.figsize"] = (5,5)
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

In [ ]:
# read in data
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X_test = pd.read_csv('X_test.csv')

# consistency in naming conventions
test_df=X_test

In [ ]:
# define training / validation 
training_df, validation_df, response_training_df, response_validation_df = train_test_split(X, y, test_size=0.2, random_state=1) 

In [ ]:
# Null Accuracy Score
y_train=response_training_df['label']
lst = [0] * len(y_train)
print(f'Null Accuracy Score: {accuracy_score(y_train, pd.Series(lst))}')

In [ ]:
# join datasets
exploration_df=X.merge(y, on='id', how='left')

In [ ]:
total_rowcount=X.shape[0]
cols=[]
missing_per=[]

for i in list(X.columns):
    
    # how many are missing (consistent with cell above)
    val=(X.loc[(X[i].isnull())].shape[0])/total_rowcount
    cols.append(i)
    missing_per.append(val)
#     print(i, val)
    
d = {'column_name':cols,'percent_missing':missing_per}
missing_df=pd.DataFrame(d)

missing_df=missing_df.loc[missing_df['percent_missing']>0]
missing_df.sort_values(by='percent_missing', ascending=False)

In [ ]:
# Checks for dupes
print('There are ', X.shape[0], ' rows in the dataset')
print('There are ', X.drop_duplicates().shape[0], ' rows after dropping duplicates')

In [ ]:
print(exploration_df.shape)

for i in list(set(exploration_df['attack_cat'])):
    
    print('*'*8)
    print(i)
    trial=exploration_df.loc[exploration_df['attack_cat']==i]
    # Are there any constant columns?
#     print('All columns:')
#     print(trial.columns[trial.nunique() >=0].to_list())

    print('\nColumns with 1 unique level')
    print(trial.columns[trial.nunique() ==1].to_list())

#     print('\nColumns with g1 unique levels')
#     print(trial.columns[trial.nunique() >1].to_list())
    print()

In [ ]:
# Partitioning using sbytes, rate, dbytes, dur, proto, smean
training_df['service_rate']=training_df.groupby('service')['rate'].transform('mean')/training_df['rate']
training_df['state_rate']=training_df.groupby('state')['rate'].transform('mean')/training_df['rate']
training_df['proto_rate']=training_df.groupby('proto')['rate'].transform('mean')/training_df['rate']
validation_df['service_rate']=validation_df.groupby('service')['rate'].transform('mean')/validation_df['rate']
validation_df['state_rate']=validation_df.groupby('state')['rate'].transform('mean')/validation_df['rate']
validation_df['proto_rate']=validation_df.groupby('proto')['rate'].transform('mean')/validation_df['rate']
test_df['service_rate']=test_df.groupby('service')['rate'].transform('mean')/test_df['rate']
test_df['state_rate']=test_df.groupby('state')['rate'].transform('mean')/test_df['rate']
test_df['proto_rate']=test_df.groupby('proto')['rate'].transform('mean')/test_df['rate']

training_df['service_sbytes']=training_df.groupby('service')['sbytes'].transform('mean')/training_df['sbytes']
training_df['state_sbytes']=training_df.groupby('state')['sbytes'].transform('mean')/training_df['sbytes']
training_df['proto_sbytes']=training_df.groupby('proto')['sbytes'].transform('mean')/training_df['sbytes']
validation_df['service_sbytes']=validation_df.groupby('service')['sbytes'].transform('mean')/validation_df['sbytes']
validation_df['state_sbytes']=validation_df.groupby('state')['sbytes'].transform('mean')/validation_df['sbytes']
validation_df['proto_sbytes']=validation_df.groupby('proto')['sbytes'].transform('mean')/validation_df['sbytes']
test_df['service_sbytes']=test_df.groupby('service')['sbytes'].transform('mean')/test_df['sbytes']
test_df['state_sbytes']=test_df.groupby('state')['sbytes'].transform('mean')/test_df['sbytes']
test_df['proto_sbytes']=test_df.groupby('proto')['sbytes'].transform('mean')/test_df['sbytes']

training_df['service_dpkts']=training_df.groupby('service')['dpkts'].transform('mean')/training_df['dpkts']
training_df['state_dpkts']=training_df.groupby('state')['dpkts'].transform('mean')/training_df['dpkts']
training_df['proto_dpkts']=training_df.groupby('proto')['dpkts'].transform('mean')/training_df['dpkts']
validation_df['service_dpkts']=validation_df.groupby('service')['dpkts'].transform('mean')/validation_df['dpkts']
validation_df['state_dpkts']=validation_df.groupby('state')['dpkts'].transform('mean')/validation_df['dpkts']
validation_df['proto_dpkts']=validation_df.groupby('proto')['dpkts'].transform('mean')/validation_df['dpkts']
test_df['service_dpkts']=test_df.groupby('service')['dpkts'].transform('mean')/test_df['dpkts']
test_df['state_dpkts']=test_df.groupby('state')['dpkts'].transform('mean')/test_df['dpkts']
test_df['proto_dpkts']=test_df.groupby('proto')['dpkts'].transform('mean')/test_df['dpkts']

training_df['service_dur']=training_df.groupby('service')['dur'].transform('mean')/training_df['dur']
training_df['state_dur']=training_df.groupby('state')['dur'].transform('mean')/training_df['dur']
training_df['proto_dur']=training_df.groupby('proto')['dur'].transform('mean')/training_df['dur']
validation_df['service_dur']=validation_df.groupby('service')['dur'].transform('mean')/validation_df['dur']
validation_df['state_dur']=validation_df.groupby('state')['dur'].transform('mean')/validation_df['dur']
validation_df['proto_dur']=validation_df.groupby('proto')['dur'].transform('mean')/validation_df['dur']
test_df['service_dur']=test_df.groupby('service')['dur'].transform('mean')/test_df['dur']
test_df['state_dur']=test_df.groupby('state')['dur'].transform('mean')/test_df['dur']
test_df['proto_dur']=test_df.groupby('proto')['dur'].transform('mean')/test_df['dur']
                                                                               
training_df['service_smean']=training_df.groupby('service')['smean'].transform('mean')/training_df['smean']
training_df['state_smean']=training_df.groupby('state')['smean'].transform('mean')/training_df['smean']
training_df['proto_smean']=training_df.groupby('proto')['smean'].transform('mean')/training_df['smean']
validation_df['service_smean']=validation_df.groupby('service')['smean'].transform('mean')/validation_df['smean']
validation_df['state_smean']=validation_df.groupby('state')['smean'].transform('mean')/validation_df['smean']
validation_df['proto_smean']=validation_df.groupby('proto')['smean'].transform('mean')/validation_df['smean']
test_df['service_smean']=test_df.groupby('service')['smean'].transform('mean')/test_df['smean']
test_df['state_smean']=test_df.groupby('state')['smean'].transform('mean')/test_df['smean']
test_df['proto_smean']=test_df.groupby('proto')['smean'].transform('mean')/test_df['smean']

In [ ]:
# feature imputation with mode value
for i in list(X.columns):
    
    print(i)
    training_df[i]=training_df[i].fillna(training_df[i].mode()[0])
    print('training_df missing %:', sum(training_df[i].isnull())/training_df[i].shape[0])
    validation_df[i]=validation_df[i].fillna(training_df[i].mode()[0])
    print('validation_df missing %:', sum(validation_df[i].isnull())/validation_df[i].shape[0])
    test_df[i]=test_df[i].fillna(training_df[i].mode()[0])
    print('test_df missing %:', sum(test_df[i].isnull())/test_df[i].shape[0], '\n')
    
#     training_df[training_df == -inf] = 0
#     validation_df[validation_df == -inf] = 0
#     test_df[test_df == -inf] = 0

In [ ]:
def categoric_binning(training_df, validation_df, test_df, categoric_column, threshold):

    # return the cumulative sum of values based on size
    cumulative_values=training_df.groupby(categoric_column)[categoric_column].size().sort_values(ascending=False)\
    .div(len(training_df)).cumsum().le(threshold)

    # filter threshold
    threshold_columns=cumulative_values[cumulative_values].index.values

    # apply to dataframe
    training_df[categoric_column+'_B']=np.where(training_df[categoric_column].isin(threshold_columns), training_df[categoric_column], 'Other')
    validation_df[categoric_column+'_B']=np.where(validation_df[categoric_column].isin(threshold_columns), validation_df[categoric_column], 'Other')
    test_df[categoric_column+'_B']=np.where(test_df[categoric_column].isin(threshold_columns), test_df[categoric_column], 'Other')
    
    # visualise value_counts()
    print(training_df[categoric_column+'_B'].value_counts())

In [ ]:
categoric_binning(training_df, validation_df, test_df, 'proto', 0.925)
categoric_binning(training_df, validation_df, test_df, 'state', 0.99)
categoric_binning(training_df, validation_df, test_df, 'service', 0.99)

In [ ]:
def concat_features(df):

    df['login_ports']=df['is_ftp_login'].astype(str)+df['is_sm_ips_ports'].astype(str)
    df['login_flow']=df['is_ftp_login'].astype(str)+df['ct_ftp_cmd'].astype(str)
    df['login_ports_flow']=df['is_ftp_login'].astype(str)+df['ct_ftp_cmd'].astype(str)+df['is_sm_ips_ports'].astype(str)

concat_features(training_df)
concat_features(validation_df)
concat_features(test_df)

In [ ]:
# Binning for sttl
bins = [0, 25, 50, 100, 200, 1000]
labels = [1, 32, 64, 128, 256]

training_df['sttl_B'] = pd.cut(training_df['sttl'], bins, labels = labels, include_lowest = True).astype(int)
validation_df['sttl_B'] = pd.cut(validation_df['sttl'], bins, labels = labels, include_lowest = True).astype(int)
test_df['sttl_B'] = pd.cut(test_df['sttl'], bins, labels = labels, include_lowest = True).astype(int)

In [ ]:
# Trans Depth Feature
training_df['trans_depth_B']=np.where(training_df['trans_depth']>0,1,0)
validation_df['trans_depth_B']=np.where(validation_df['trans_depth']>0,1,0)
test_df['trans_depth_B']=np.where(test_df['trans_depth']>0,1,0)

In [ ]:
# Total packages features
training_df['total_packets_FE']=training_df['dur']*training_df['rate']
validation_df['total_packets_FE']=validation_df['dur']*validation_df['rate']
test_df['total_packets_FE']=test_df['dur']*test_df['rate']

In [ ]:
def mean_encoder(training_df, validation_df, test_df, col, response):

    """ function to mean encode categorical features 
        any missing values are imputed with mode """

    # Create dictionary
    mean_encoding=training_df.groupby(col)[response].mean().to_dict()

    # Apply to train
    training_df[col+'_ME']=training_df[col].replace(mean_encoding)
    training_df[col+'_ME']=training_df[col+'_ME'].fillna(training_df[col+'_ME'].mode()[0])

    # Apply to valid
    validation_df[col+'_ME']=validation_df[col].replace(mean_encoding)
    validation_df[col+'_ME']=validation_df[col+'_ME'].fillna(validation_df[col+'_ME'].mode()[0])

    # Apply to test
    test_df[col+'_ME']=test_df[col].replace(mean_encoding)
    test_df[col+'_ME']=test_df[col+'_ME'].fillna(test_df[col+'_ME'].mode()[0])

In [ ]:
# One Hot Encoding and Mean encoding
num_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
fe_categorical_columns = list(training_df.select_dtypes(exclude=num_dtypes).columns)
unwanted = ['proto', 'state', 'service']
fe_categorical_columns = [ele for ele in fe_categorical_columns if ele not in unwanted]

training_df=training_df.merge(response_training_df, on='id', how='left')

for i in fe_categorical_columns:
    
    print(i)
    
    mean_encoder(training_df, validation_df, test_df, i, 'label')
    
    # Get one hot encoding of columns B
    one_hot_train = pd.get_dummies(training_df[i], prefix=i)
    one_hot_valid = pd.get_dummies(validation_df[i], prefix=i)
    one_hot_test = pd.get_dummies(test_df[i], prefix=i)
    
    # Drop column B as it is now encoded
    training_df = training_df.drop(i,axis = 1)
    validation_df = validation_df.drop(i,axis = 1)
    test_df = test_df.drop(i,axis = 1)
    
    # Join the encoded df
    training_df = training_df.join(one_hot_train)
    validation_df = validation_df.join(one_hot_valid)
    test_df = test_df.join(one_hot_test)

In [ ]:
# remove outliers
def outlier_capping(training_df, validation_df, test_df, variable, multiplier):

    ''' cap and collar the response variable '''

    q1 = np.percentile(training_df[variable],25)
    q3 = np.percentile(training_df[variable],75)
    iqr = q3 - q1
    lower = q1 - (iqr * multiplier)
    upper = q3 + (iqr * multiplier)

    # adjust on train
    training_df[variable] = np.where(training_df[variable]<=lower, lower, training_df[variable])
    training_df[variable] = np.where(training_df[variable]>=upper, upper, training_df[variable])

    # adjust on train
    validation_df[variable] = np.where(validation_df[variable]<=lower, lower, validation_df[variable])
    validation_df[variable] = np.where(validation_df[variable]>=upper, upper, validation_df[variable])

    # adjust on train
    test_df[variable] = np.where(test_df[variable]<=lower, lower, test_df[variable])
    test_df[variable] = np.where(test_df[variable]>=upper, upper, test_df[variable])

In [ ]:
# cap all numeric values which arent integers
s=(training_df.dtypes==float)

for i in s[s].keys():
    
    print(i)
    outlier_capping(training_df, validation_df, test_df, i, 1.5)

In [ ]:
# list all columns
training_columns=list(training_df.columns)
validation_columns=list(validation_df.columns)
test_columns=list(test_df.columns)

# intersection of all common items in train, valid, test
modelling_cols=list(set(training_columns).intersection(validation_columns, test_columns))

unwanted = ['proto', 'state', 'service', 'id']
modelling_cols = [ele for ele in modelling_cols if ele not in unwanted]

In [ ]:
# create response variables
for i in list(set(response_training_df['attack_cat'])):
    
    # create response cols
    response_training_df[i]=np.where(response_training_df['attack_cat']==i,1,0)
    response_validation_df[i]=np.where(response_validation_df['attack_cat']==i,1,0)

In [ ]:
def fit_algo(algo, X_train, Y_train, X_valid, Y_valid, cv):
    
    # fit the model
    model = algo.fit(X_train, Y_train)
    
    # check its score
    acc = round(model.score(X_train, Y_train) *100, 2)
    f1_train = f1_score(Y_train, model.predict(X_train))
    f1_test = f1_score(Y_valid, model.predict(X_valid))
    print('Training F1 Score:', f1_train)
    print('Test F1 Score:', f1_test)    
    
    y_pred = model_selection.cross_val_predict(algo, X_train, Y_train, cv=cv, n_jobs = -1)
    y_pred_test = model_selection.cross_val_predict(algo, X_valid, Y_valid, cv=cv, n_jobs = -1)
    f1_train_cv = f1_score(Y_train, y_pred)
    f1_test_cv = f1_score(Y_valid, y_pred_test)
    print('Cross Validated Training F1 Score:', f1_train_cv)
    print('Cross Validated Test F1 Score:', f1_test_cv) 
    
    return model

def z_score(df, col):
    
    return (df[col]-df[col].mean())/df[col].std()

def min_max(df, col):
    
    return (df[col]-df[col].min())/(df[col].max()-df[col].min())

In [ ]:
i='Normal'

print('model #9:', i)

# define modelling cols and response variable for Exploits
X_train=training_df[modelling_cols]
Y_train=response_training_df[i]
X_valid=validation_df[modelling_cols]
Y_valid=response_validation_df[i]

# RandomForestClassifier
clf=RandomForestClassifier(n_estimators = 125, criterion = 'entropy')
rf_Normal = fit_algo(clf, X_train, Y_train, X_valid, Y_valid, 3)

# Score on test_df
training_df[i]=rf_Normal.predict_proba(training_df[modelling_cols])[:,0]
training_df[i+'_z']=z_score(training_df, i)

validation_df[i]=rf_Normal.predict_proba(validation_df[modelling_cols])[:,0]
validation_df[i+'_z']=z_score(validation_df, i)

test_df[i]=rf_Normal.predict_proba(test_df[modelling_cols])[:,0]
test_df[i+'_z']=z_score(test_df, i)

training_df[i+'_m']=min_max(training_df, i)
validation_df[i+'_m']=min_max(validation_df, i)
test_df[i+'_m']=min_max(test_df, i)

In [ ]:
training_df['classification_prediction']=training_df['Normal']
validation_df['classification_prediction']=validation_df['Normal']
test_df['classification_prediction']=test_df['Normal']

test_df['classification_prediction']=np.where(test_df['classification_prediction']>test_df['classification_prediction'].mean(),1,0)

In [ ]:
"""Multi-Class modelling
"""

# feature selection
mc_X_train=training_df[modelling_cols]
mc_Y_train=response_training_df['attack_cat']
mc_X_valid=validation_df[modelling_cols]
mc_Y_valid=response_validation_df['attack_cat']

"""# transform based on submission criteria
response_training_df['attack_cat']=response_training_df['attack_cat'].replace(['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode','Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic'],
[0,1,2,3,4,5,6,7,8,9])response_validation_df['attack_cat']=response_validation_df['attack_cat'].replace(['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode','Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic'],
[0,1,2,3,4,5,6,7,8,9])"""

from sklearn.linear_model import Perceptron
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix# Create the SVM

rfc = RandomForestClassifier(n_estimators = 140, criterion = 'entropy', class_weight='balanced')# Make it an OvR classifier
ovr_classifier = OneVsRestClassifier(rfc)# Fit the data to the OvR classifier
ovr_classifier = ovr_classifier.fit(mc_X_train, mc_Y_train)# Evaluate by means of a confusion matrix
matrix = plot_confusion_matrix(ovr_classifier, mc_X_valid, mc_Y_valid,
cmap=plt.cm.Blues,
normalize='true')
plt.title('Confusion matrix for OvR classifier')
plt.show(matrix)
plt.show()

In [ ]:
test_df['classification_prediction']=np.where(test_df['classification_prediction']>test_df['classification_prediction'].mean(),1,0)
test_df['output_classes']=ovr_classifier.predict(test_df[modelling_cols])
test_df['attack_cat_prediction']=test_df['output_classes'].replace(['Normal', 'Backdoor', 'Analysis', 'Fuzzers', 'Shellcode','Reconnaissance', 'Exploits', 'DoS', 'Worms', 'Generic'],
                                     [0,1,2,3,4,5,6,7,8,9])

In [ ]:
# score xgb_model on test
y_test_id = test_df['id']
y_test_label_pred = test_df['classification_prediction']
y_test_attack_cat_pred = test_df['attack_cat_prediction']

# export for submission
y_pred = pd.DataFrame(data = {'id': y_test_id, 'label' : y_test_label_pred, 'attack_cat' : y_test_attack_cat_pred})
y_pred.to_csv('output.csv', index=False)